## Example 1: Finding CRISPR-cas gene clusters in a cyanobacteria genome

In this example, we will annotate and visualize CRISPR-cas gene clusters in the cyanobacteria species Rippkaea orientalis. CRISPR-cas is a widespread bacterial defense system, found in over 50% of all known prokaryotic species. Notably, it has been repurposed as an efficient gene editing tool that has become ubiquitous in molecular biology workflows. R. orientalis was selected because it's genome contains two complete CRISPR-cas loci (one chromosomal, and one extrachromosomal/plasmid).

You can download the complete assembled genome [here](https://www.ncbi.nlm.nih.gov/assembly/GCF_000024045.1/); it is also available at https://github.com/alexismhill3/Opfi in the `tutorials` folder (`data/GCF_000024045.1_ASM2404v1_genomic.fna.gz`). This tutorial assumes that the user has already cloned and built Opfi, installed NCBI BLAST+ **and** PILER-CR ([ZZZ link to the docs page about Ofpi installation]), and is using `tutorials` as the working directory. Some familiarity with BLAST and the basic homology search algorithm may also be helpful, but is not required. 

**1. Use the makeblastdb utility to convert a cas protein database to BLAST format**  
We start by converting our cas sequence database to a format that BLAST can recognize. We can use the command line utility `makeblastdb`, which is provided as part of the NCBI BLAST+ distribution. A set of ~20,000 non-redundant cas sequences, downloaded from [Uniprot](https://www.uniprot.org/uniref/) is available as a tar archive (`tutorials/data/cas_database.tar.gz`). We'll make a new directory, "blastdb", and extract sequences there:

In [37]:
! mkdir blastdb
! cd blastdb && tar -xzf ../data/cas_database.tar.gz && cd ..

Next, we'll create two BLAST databases with the sequence data: one containing cas1 sequences only, and another that contains the remaining cas sequences.

In [22]:
! cd blastdb && cat cas1.fasta | makeblastdb -dbtype prot -title cas1 -hash_index -out cas1_db && cd ..
! cd blastdb && cat cas[2-9].fasta cas1[0-2].fasta casphi.fasta | makeblastdb -dbtype prot -title cas_all -hash_index -out cas_all_but_1_db



Building a new DB, current time: 05/26/2021 13:42:55
New DB name:   /home/alexis/projects/Opfi/tutorials/blastdb/cas1_db
New DB title:  cas1
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 2156 sequences in 0.0846369 seconds.


Building a new DB, current time: 05/26/2021 13:42:55
New DB name:   /home/alexis/projects/Opfi/tutorials/blastdb/cas_all_but_1_db
New DB title:  cas_all
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 20561 sequences in 0.757788 seconds.


`-dbtype prot` simply tells `makeblastdb` to expect amino acid sequences. We use `-title` and `-out` to name the database (required by BLAST) and to prefix the database files, respectively. `-hash_index` directs `makeblastdb` to generate a hash index of protein sequences, which can speed up computation time.

**2. Use Opfi's Gene Finder module to search for CRISPR-cas loci**

CRISPR-cas systems are quite diverse, with Cas1 (an endonuclease) being the only protein family common to the vast majority of CRISPR systems. We will use cas1 to define candidate CRISPR-cas loci in the R. orientalis genome, and then search for additional CRISPR-cas features in the vicinity of putative cas1 genes.

First, create another directory for output:

In [66]:
! mkdir example_1_output

The following bit of code uses Opfi's Gene Finder module to search for CRISPR-cas systems:

In [1]:
from gene_finder.pipeline import Pipeline
import os

genomic_data = "data/GCF_000024045.1_ASM2404v1_genomic.fna.gz"
output_directory = "example_1_output"

p = Pipeline()
p.add_seed_step(db="blastdb/cas1_db", name="cas1", e_val=0.001, blast_type="PROT", num_threads=1)
p.add_filter_step(db="blastdb/cas_all_but_1_db", name="cas_all", e_val=0.001, blast_type="PROT", num_threads=1)
p.add_crispr_step()

# use the input filename as the job id
# results will be written to the file <job id>_results.csv
job_id = os.path.basename(genomic_data)
results = p.run(job_id=job_id, data=genomic_data, output_directory=output_directory, min_prot_len=90, span=10000, gzip=True)

FileNotFoundError: [Errno 2] No such file or directory: 'blastp'

First, we initialize a `Pipeline` object. Internally, this keeps a master list of all search parameters, and also keeps track of the systems that meet initial search criteria. Next, we add three search steps to the pipeline:

1. `add_seed_step` Use BLAST to search the input genome against a database of cas1 sequences. Regions around putative cas1 hits become the intial candidates, and the rest of the genome is ignored.
2. `add_filter_step` Search candidate regions against remaining cas sequences. Candidates without at least one additional putative cas gene are also discarded. 
3. `add_crispr_step` Use PILER-CR to look for CRISPR repeat sequences in any remaining candidate loci.  

Finally, we run the pipeline, executing steps in the order they we added. `min_prot_len` sets the minimum length (in amino acid residues) of hits to keep. `span` is the region directly up- or downstream of initial hits. So, each candidate system will be about 20 kbp in length. Results are written to a single CSV file (you can read more about the output format [here](ZZZ output format page in docs)). Final candidate loci contain at least one putative cas1 gene and one additional cas gene. As we will see, this relatively permissive criteria captures some non-CRISPR-cas loci. Opfi has additional modules for reducing unlikely systems.

**3. Visualize annotated CRISPR-cas gene clusters using Opfi's Operon Analyzer module**

It is sometimes useful to visualize candidate systems, especially during the exploratory phase of a genomics survey. Opfi provides a few functions for visualizing candidate systems as gene diagrams. We'll use these to visualize the CRISPR-cas gene clusters in R. orientalis:

In [19]:
import csv
import sys
from operon_analyzer import load, visualize

feature_colors = { "cas1": "lightblue",
                    "cas2": "seagreen",
                    "cas3": "gold",
                    "cas4": "springgreen",
                    "cas5": "darkred",
                    "cas6": "thistle",
                    "cas7": "coral",
                    "cas8": "red",
                    "cas9": "palegreen",
                    "cas10": "yellow",
                    "cas11": "tan",
                    "cas12": "orange",
                    "cas13": "saddlebrown",
                    "casphi": "olive",
                    "CRISPR array": "purple"
                    }

# read in the output from Gene Finder and create a gene diagram for each cluster (operon)
with open("example_1_output/GCF_000024045.1_ASM2404v1_genomic.fna.gz_results.csv", "r") as operon_data:
    operons = load.load_operons(operon_data)
    visualize.plot_operons(operons=operons, output_directory="example_1_output", feature_colors=FEATURE_COLORS, nucl_per_line=25000)

We successfully identified the two CRISPR-cas systems in R. orientalis. We also found some systems that don't look like CRISPR-cas operons. Since we used a relatively permissive e-value threshhold of 0.001 when running BLAST, Opfi retained regions with very low sequence similarity to true CRISPR-cas genes. In fact, these regions are likely not CRISPR-cas loci at all. Using a lower e-value would likely eliminate these "false positive" systems, but Opfi also has additional functions for filtering out unlikely candidates _after_ the intial BLAST search. 

In general, we have found that using permissive BLAST parameters intially, and then filtering or eliminating candidates later, is the most effective way to search for novel gene clusters in large amounts of genomic/metagenomic data. In this toy example, we could re-run BLAST many times without significant cost. But on a more realistic dataset, needing to re-do the computationally expensive homology search could severely detrail a project. It is better to cast a wide net first and then narrow results later.

Finally, clean up the temporary directories, if desired:

In [1]:
! rm -r example_1_output blastdb

## Example 2: Filter CRISPR-cas systems according to genomic composition

In [68]:
! mkdir example_2_output

In [69]:
from operon_analyzer import analyze, rules


fs = rules.FilterSet().pick_overlapping_features_by_bit_score(0.9)
cas_types = ["I", "II", "III", "V"]

rulesets = []
# type I rules
rulesets.append(rules.RuleSet().contains_group(feature_names = ["cas5", "cas7"], max_gap_distance_bp = 1000, require_same_orientation = True) \
                               .require("cas3"))
# type II rules
rulesets.append(rules.RuleSet().contains_at_least_n_features(feature_names = ["cas1", "cas2", "cas9"], feature_count = 3) \
                               .minimum_size("cas9", 3000))
# type III rules
rulesets.append(rules.RuleSet().contains_group(feature_names = ["cas5", "cas7"], max_gap_distance_bp = 1000, require_same_orientation = True) \
                               .require("cas10"))
# type V rules
rulesets.append(rules.RuleSet().contains_at_least_n_features(feature_names = ["cas1", "cas2", "cas12"], feature_count = 3))

for rs, cas_type in zip(rulesets, cas_types):
    with open("data/refseq_fusobacteria.csv", "r") as input_csv:
        with open(f"example_2_output/refseq_fuso_filtered_type{cas_type}.csv", "w") as output_csv:
            analyze.evaluate_rules_and_reserialize(input_csv, rs, fs, output_csv)

In [70]:
import csv
import sys
from operon_analyzer import load, visualize

feature_colors = { "cas1": "lightblue",
                    "cas2": "seagreen",
                    "cas3": "gold",
                    "cas4": "springgreen",
                    "cas5": "darkred",
                    "cas6": "thistle",
                    "cas7": "coral",
                    "cas8": "red",
                    "cas9": "palegreen",
                    "cas10": "yellow",
                    "cas11": "tan",
                    "cas12": "orange",
                    "cas13": "saddlebrown",
                    "casphi": "olive",
                    "CRISPR array": "purple"
                    }

# read in the output from Gene Finder and create a gene diagram for each cluster (operon)
with open("example_2_output/refseq_fuso_filtered_typeV.csv", "r") as operon_data:
    operons = load.load_operons(operon_data)
    visualize.plot_operons(operons=operons, output_directory="example_2_output", feature_colors=feature_colors, nucl_per_line=25000)

In [2]:
! rm -r example_2_output